# Testing

In this notebook you can explore and test the resulting database tables from our Dagster ETL process.
Here you can analyze


In [51]:
import duckdb
import polars as pl
from IPython import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np
import pandas as pd

### Create Database Connection


In [52]:
%load_ext sql
conn = duckdb.connect(database="../dsp-dagster/data_systems_project.duckdb")
%sql conn --alias duckdb
%sql SHOW ALL TABLES;

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Running query in 'duckdb'

database,schema,name,column_names,column_types,temporary
data_systems_project,cleaned,cleaned_knmi_weather_data,"['Station_code', 'Date', 'Hour', 'Dd', 'Fh', 'Ff', 'Fx', 'T', 'T10n', 'Td', 'Sq', 'Q', 'Dr', 'Rh', 'P', 'Vv', 'N', 'U', 'Ww', 'Ix', 'M', 'R', 'S', 'O', 'Y']","['BIGINT', 'DATE', 'TINYINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT']",False
data_systems_project,cleaned,cleaned_storm_incidents,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Starttime_Hour', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute']","['BIGINT', 'DATE', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT']",False
data_systems_project,joined,incident_deployments,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Starttime_Hour', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute', 'Deployment_ID', 'Vehicle_Type', 'Vehicle_Role', 'Fire_Station', 'Fire_Station_Service_Status', 'Driving_Time_To_Incident']","['BIGINT', 'DATE', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR']",False
data_systems_project,joined,incident_deployments_vehicles,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Starttime_Hour', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute', 'Deployment_ID', 'Vehicle_Type', 'Vehicle_Role', 'Fire_Station', 'Fire_Station_Service_Status', 'Driving_Time_To_Incident', 'Vehicle']","['BIGINT', 'DATE', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR']",False
data_systems_project,joined,incident_deployments_vehicles_weather,"['Station_code', 'Date', 'Hour', 'Dd', 'Fh', 'Ff', 'Fx', 'T', 'T10n', 'Td', 'Sq', 'Q', 'Dr', 'Rh', 'P', 'Vv', 'N', 'U', 'Ww', 'Ix', 'M', 'R', 'S', 'O', 'Y', 'Incident_ID', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LAT', 'Incident_Endtime_Hour', 'Incident_Duration_Hour', 'Incident_Starttime_Minute', 'Incident_Endtime_Minute', 'Incident_Duration_Minute', 'Deployment_ID', 'Vehicle_Type', 'Vehicle_Role', 'Fire_Station', 'Fire_Station_Service_Status', 'Driving_Time_To_Incident', 'Vehicle']","['BIGINT', 'DATE', 'TINYINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'BIGINT', 'TIME', 'TIME', 'TIME', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'TINYINT', 'BIGINT', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR']",False
data_systems_project,joined,incident_deployments_vehicles_wijken,"['Incident_ID', 'Date', 'Incident_Starttime', 'Incident_Endtime', 'Incident_Duration', 'Incident_Priority', 'Service_Area', 'Municipality', 'Damage_Type', 'LON', 'LA

##### Drop Tables


In [53]:
# %sql DROP TABLE public.cbs_buurten;
# %sql DROP TABLE public.cbs_wijken;

##### Retrieve Tables as Polars DataFrame


In [54]:
cbs_wijken = conn.execute(
    """
    SELECT * FROM public.cbs_wijken """
).df()

service_areas = conn.execute(
    """
    SELECT * FROM public.service_areas """
).df()

incident_deployments_vehicles = conn.execute(
    """
    SELECT * FROM joined.incident_deployments_vehicles"""
).df()


cleaned_knmi_weather_data = conn.execute(
    """
    SELECT * FROM cleaned.cleaned_knmi_weather_data
    """
).df()


incident_deployments_vehicles_wijken = conn.execute(
    """
    SELECT * FROM joined.incident_deployments_vehicles_wijken
    """
).df()

incident_deployments_vehicles_weather = conn.execute(
    """
    SELECT * FROM joined.incident_deployments_vehicles_weather
    """
).df()


# Close the database connection
conn.close()

In [55]:
# cbs_wijken.filter(
#     [
#         pl.col("gemeentenaam") == "Amsterdam",
#     ]
# ).head()

In [56]:
# incident_deployments_vehicles_wijken.head()

In [57]:
incident_deployments_vehicles_weather.head()

,Station_code,Date,Hour,Dd,Fh,Ff,Fx,T,T10n,Td,...,Incident_Starttime_Minute,Incident_Endtime_Minute,Incident_Duration_Minute,Deployment_ID,Vehicle_Type,Vehicle_Role,Fire_Station,Fire_Station_Service_Status,Driving_Time_To_Incident,Vehicle
0,240,2005-01-01,1,260,40,30,60,68,NaN,57,...,NaN,NaN,NaN,NaN,None,None,None,None,None,None
1,240,2005-01-01,2,230,30,30,60,65,NaN,52,...,NaN,NaN,NaN,NaN,None,None,None,None,None,None
2,240,2005-01-01,3,230,40,30,50,43,NaN,34,...,NaN,NaN,NaN,NaN,None,None,None,None,None,None
3,240,2005-01-01,4,220,40,40,50,38,NaN,32,...,NaN,NaN,NaN,NaN,None,None,None,None,None,None
4,240,2005-01-01,5,230,40,40,50,38,NaN,34,...,NaN,NaN,NaN,NaN,None,None,None,None,None,None


In [58]:
incident_deployments_vehicles_weather.head()
combined_df = incident_deployments_vehicles_weather

In [59]:
combined_df["Incident"] = np.where(combined_df["Incident_ID"].notna(), 1, 0)
combined_df["Incident"].unique()

array([0, 1])

In [60]:
combined_df.fillna(0, inplace=True)
# print(combined_df.dtypes)
# print(combined_df.dtypes.iloc[25:47])

In [61]:
categorical_columns = [
    "Incident_Starttime",
    "Incident_Endtime",
    "Incident_Duration",
    "Service_Area",
    "Municipality",
    "Damage_Type",
    "Vehicle_Type",
    "Vehicle_Role",
    "Fire_Station",
    "Fire_Station_Service_Status",
    "Driving_Time_To_Incident",
    "Vehicle",
]
combined_df[categorical_columns] = combined_df[categorical_columns].astype("category")

In [62]:
combined_df["Vehicle"].unique()

[0, 'Tankautospuit', 'HV met kraan', 'Autoladder', 'ON Blusrobot', ..., 'Bijzondere Blusmiddelen HAB', 'Haakarmvoertuig met kraan', 'Haakarmvoertuig', 'Waterongevallenvoertuig', 'Waterongevallen Vaartuig']
Length: 12
Categories (12, object): [0, 'Autoladder', 'BU Blusrobot elke capaciteit', 'Bijzondere Blusmiddelen HAB', ..., 'Schuimblusvoertuig', 'Tankautospuit', 'Waterongevallen Vaartuig', 'Waterongevallenvoertuig']

In [65]:
# Prepare your data
X = combined_df.drop(columns=["Incident", "Date"])
y = combined_df["Incident"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Convert target variables to integers
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Train XGBoost model
model = xgb.XGBClassifier(
    use_label_encoder=False, enable_categorical=True, eval_metric="logloss"
)
model.fit(X_train, y_train)

# Evaluate the model
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy}")

# Feature Importance
feature_importances = model.feature_importances_
features = X.columns
importance_df = pd.DataFrame({"Feature": features, "Importance": feature_importances})
importance_df = importance_df.sort_values(by="Importance", ascending=False)
print(importance_df)

Model Accuracy: 1.0
                        Feature  Importance
25           Incident_Starttime    0.999630
5                            Fx    0.000175
3                            Fh    0.000126
4                            Ff    0.000053
13                            P    0.000016
0                  Station_code    0.000000
35       Incident_Duration_Hour    0.000000
27            Incident_Duration    0.000000
28            Incident_Priority    0.000000
29                 Service_Area    0.000000
30                 Municipality    0.000000
31                  Damage_Type    0.000000
32                          LON    0.000000
33                          LAT    0.000000
34        Incident_Endtime_Hour    0.000000
37      Incident_Endtime_Minute    0.000000
36    Incident_Starttime_Minute    0.000000
38     Incident_Duration_Minute    0.000000
39                Deployment_ID    0.000000
40                 Vehicle_Type    0.000000
41                 Vehicle_Role    0.000000
42          

In [69]:
predictions

array([0, 0, 0, ..., 0, 0, 0])